# 2 PTT计算准备

## 步骤 2.1：数据加载与检查

In [3]:
import pandas as pd
import os

output_dir = 'D:\code\Python\PI_Lab\output\csv_output'
experiment_id = '1'  # 替换为具体实验ID

# 加载Biopac数据（血压）
biopac_file = f'{experiment_id}_biopac_aligned.csv'
biopac_df = pd.read_csv(os.path.join(output_dir, biopac_file))
print(f"Biopac数据列: {biopac_df.columns.tolist()}")
print(f"样本数: {len(biopac_df)}")

# 加载HUB数据
sensor_positions = {'sensor2': 'nose', 'sensor3': 'finger', 'sensor4': 'wrist', 'sensor5': 'ear'}
hub_data = {}
for sensor, position in sensor_positions.items():
    hub_file = f'{experiment_id}_hub_{sensor}_aligned.csv'
    hub_df = pd.read_csv(os.path.join(output_dir, hub_file))
    hub_data[sensor] = hub_df
    print(f"{position} ({sensor}) 数据列: {hub_df.columns.tolist()}")
    print(f"样本数: {len(hub_df)}")

Biopac数据列: ['timestamp', 'bp', 'cardiac_index', 'cardiac_output', 'diastolic_bp', 'hr', 'mean_bp', 'rsp', 'systemic_vascular_resistance', 'systolic_bp']
样本数: 66669
nose (sensor2) 数据列: ['timestamp', 'red', 'ir', 'green', 'ax', 'ay', 'az', 'rx', 'ry', 'rz', 'mx', 'my', 'mz', 'temp', 'time']
样本数: 66669
finger (sensor3) 数据列: ['timestamp', 'red', 'ir', 'green', 'ax', 'ay', 'az', 'rx', 'ry', 'rz', 'mx', 'my', 'mz', 'temp', 'time']
样本数: 66669
wrist (sensor4) 数据列: ['timestamp', 'red', 'ir', 'green', 'ax', 'ay', 'az', 'rx', 'ry', 'rz', 'mx', 'my', 'mz', 'temp', 'time']
样本数: 66669
ear (sensor5) 数据列: ['timestamp', 'red', 'ir', 'green', 'ax', 'ay', 'az', 'rx', 'ry', 'rz', 'mx', 'my', 'mz', 'temp', 'time']
样本数: 66669


## 相关性

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
PTT与真实血压的相关性分析脚本
- 加载ptt_windowed_exp_1.csv和1_biopac_aligned.csv
- 匹配有效窗口的PTT与对应时间段的血压值
- 计算皮尔逊相关系数并可视化
"""

import os
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns

# 设置路径
data_path = r"D:\code\Python\PI_Lab\output\csv_output"
ptt_path = r"D:\code\Python\PI_Lab\ptt_output2\exp_1\ptt_windowed_exp_1.csv"
biopac_path = os.path.join(data_path, "1_biopac_aligned.csv")
output_dir = r"D:\code\Python\PI_Lab\ptt_output2\exp_1"
os.makedirs(output_dir, exist_ok=True)

# 加载数据
print("📂 加载数据...")
ptt_df = pd.read_csv(ptt_path)
biopac_df = pd.read_csv(biopac_path)

# 检查数据
print(f"PTT 数据: {len(ptt_df)} 行, 列: {list(ptt_df.columns)}")
print(f"Biopac 数据: {len(biopac_df)} 行, 列: {list(biopac_df.columns)}")

# 假设 Biopac 数据包含 'timestamp' 和 'bp' 列
# 检查 Biopac 数据是否包含 'bp' 列
if 'bp' not in biopac_df.columns:
    raise ValueError("Biopac 数据中缺少 'bp' 列，请检查数据！")

# 提取有效窗口的 PTT 数据
valid_ptt = ptt_df.dropna(subset=['ptt_ms']).copy()
print(f"有效 PTT 数据: {len(valid_ptt)} 行")

# 匹配窗口内的血压值
bp_values = []
for idx, row in valid_ptt.iterrows():
    start_time = row['window_start_s']
    end_time = row['window_end_s']
    
    # 找到对应时间窗口的 Biopac 数据
    mask = (biopac_df['timestamp'] >= start_time) & (biopac_df['timestamp'] <= end_time)
    window_bp = biopac_df[mask]['bp']
    
    # 计算窗口内的平均血压值
    if len(window_bp) > 0:
        mean_bp = window_bp.mean()
    else:
        mean_bp = np.nan  # 如果窗口内没有数据，标记为 NaN
    
    bp_values.append(mean_bp)

# 将平均血压值添加到 PTT 数据框
valid_ptt['mean_bp'] = bp_values

# 去除 NaN 值
valid_data = valid_ptt.dropna(subset=['ptt_ms', 'mean_bp'])
print(f"匹配后的有效数据: {len(valid_data)} 行")

# 计算皮尔逊相关系数
if len(valid_data) > 1:
    correlation, p_value = pearsonr(valid_data['ptt_ms'], valid_data['mean_bp'])
    print(f"📊 皮尔逊相关系数: {correlation:.3f}, p-value: {p_value:.3e}")
else:
    print("⚠️ 有效数据不足，无法计算相关性！")
    correlation, p_value = np.nan, np.nan

# 可视化
plt.figure(figsize=(10, 6))
sns.scatterplot(data=valid_data, x='ptt_ms', y='mean_bp', hue='sensor_pair', size='sensor_pair', alpha=0.6)
plt.xlabel('PTT (ms)')
plt.ylabel('Mean Blood Pressure (mmHg)')
plt.title(f'PTT vs Blood Pressure Correlation (r={correlation:.3f}, p={p_value:.3e})')
plt.grid(True, alpha=0.3)
plt.legend(title='Sensor Pair', bbox_to_anchor=(1.05, 1), loc='upper left')

# 保存图形
plot_file = os.path.join(output_dir, "ptt_bp_correlation_exp_1.png")
plt.tight_layout()
plt.savefig(plot_file, dpi=300, bbox_inches='tight')
plt.close()
print(f"💾 保存相关性图: {plot_file}")

# 保存相关性结果
results = {
    'correlation': correlation,
    'p_value': p_value,
    'num_valid_samples': len(valid_data),
    'sensor_pairs': valid_data['sensor_pair'].unique().tolist()
}
results_df = pd.DataFrame([results])
results_file = os.path.join(output_dir, "ptt_bp_correlation_summary_exp_1.csv")
results_df.to_csv(results_file, index=False)
print(f"💾 保存相关性汇总: {results_file}")

print("✅ 相关性分析完成！")

📂 加载数据...
PTT 数据: 2774 行, 列: ['heartbeat_id', 'window_id', 'sensor_pair', 'sensor_names', 'ptt_ms', 'sensor2_time_s', 'sensor3_time_s', 'window_start_s', 'window_end_s', 'sensor4_time_s', 'sensor5_time_s']
Biopac 数据: 66669 行, 列: ['timestamp', 'bp', 'cardiac_index', 'cardiac_output', 'diastolic_bp', 'hr', 'mean_bp', 'rsp', 'systemic_vascular_resistance', 'systolic_bp']
有效 PTT 数据: 2774 行


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


匹配后的有效数据: 0 行
⚠️ 有效数据不足，无法计算相关性！
💾 保存相关性图: D:\code\Python\PI_Lab\ptt_output2\exp_1\ptt_bp_correlation_exp_1.png
💾 保存相关性汇总: D:\code\Python\PI_Lab\ptt_output2\exp_1\ptt_bp_correlation_summary_exp_1.csv
✅ 相关性分析完成！


## 4. 血压预测模型构建

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 准备特征和标签
X = []  # PTT特征
y_systolic = biopac_df['systolic_bp'].values
y_diastolic = biopac_df['diastolic_bp'].values

for sensor, df in hub_data.items():
    if sensor != reference_sensor:
        for channel in ['red', 'ir', 'green']:
            if f'{channel}_peaks' in df.columns and f'{channel}_peaks' in ref_df.columns:
                ref_peaks = np.where(ref_df[f'{channel}_peaks'] == 1)[0]
                target_peaks = np.where(df[f'{channel}_peaks'] == 1)[0]
                ptt = calculate_ptt(ref_peaks, target_peaks, timestamps)
                X.append(ptt)

X = np.array(X).reshape(-1, 1)

# 拆分训练集和测试集
X_train, X_test, y_s_train, y_s_test = train_test_split(X, y_systolic, test_size=0.2, random_state=42)
X_train, X_test, y_d_train, y_d_test = train_test_split(X, y_diastolic, test_size=0.2, random_state=42)

# 训练模型
model_systolic = LinearRegression()
model_systolic.fit(X_train, y_s_train)
y_s_pred = model_systolic.predict(X_test)
print(f"收缩压 MSE: {mean_squared_error(y_s_test, y_s_pred):.2f}")

model_diastolic = LinearRegression()
model_diastolic.fit(X_train, y_d_train)
y_d_pred = model_diastolic.predict(X_test)
print(f"舒张压 MSE: {mean_squared_error(y_d_test, y_d_pred):.2f}")

ValueError: Found input variables with inconsistent numbers of samples: [0, 66669]